In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b85630dc-2deb-43cf-bdc1-4af0708b4a9c)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 4, Finished, Available, Finished)

In [3]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
.setTextCol("description")
.setKind ("SentimentAnalysis")
.setOutputCol("response")
.setErrorCol("error"))

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 399fb702-8c55-45e7-9691-b72363724d84)

In [6]:
#Create a sentiment column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 8, Finished, Available, Finished)

In [7]:
display(sentiment_df)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a4aadf38-1a90-4167-9f3d-160fc61712be)

In [8]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df_final)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f25c7596-ddd7-49cc-9e76-9853e75fb54e)

In [10]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    # Write DataFrame as a Delta table
    sentiment_df_final.write.format("delta").saveAsTable(table_name)
    
except AnalysisException:

    print("Table Already Exists")

    # Create or replace a temporary view
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    # Perform the MERGE operation
    spark.sql(f"""
        MERGE INTO {table_name} AS target_table
        USING vw_sentiment_df_final AS source_view
        ON source_view.url = target_table.url
        WHEN MATCHED AND (
            source_view.title != target_table.title OR
            source_view.description != target_table.description OR
            source_view.category != target_table.category OR
            source_view.image != target_table.image OR
            source_view.provider != target_table.provider OR
            source_view.datePublished != target_table.datePublished
        )
        THEN UPDATE SET *

        WHEN NOT MATCHED THEN INSERT *
           
    """)

StatementMeta(, a9a54edb-dccb-413f-a53b-53576c05012b, 12, Finished, Available, Finished)